# ***Imports***

In [ ]:
import os
from pyspark.sql.functions import count
import pandas as pd
from scipy import stats

# **Loading a Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile

# Path to the archive file
archive_path = '/content/drive/My Drive/MMU/Assignments/HPC/archive.zip'

# Define the folder to extract the contents
extracted_folder_path = '/content/drive/My Drive/MMU/Assignments/HPC/extracted_data/'

# Extract the ZIP file
with zipfile.ZipFile(archive_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print("Extraction complete.")


Extraction complete.


In [ ]:
# List the extracted files
extracted_files = os.listdir(extracted_folder_path)
extracted_files


['ratings_Electronics (1).csv']

**Load the CSV File**

In [ ]:
import pandas as pd

# Path to the CSV file
csv_path = '/content/drive/My Drive/MMU/Assignments/HPC/extracted_data/ratings_Electronics (1).csv'

# Load the CSV file into a Pandas DataFrame
df = pd.read_csv(csv_path)

# Display the first few rows of the DataFrame
df.head()


,AKM1MP6P0OYPR,0132793040,5.0,1365811200
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600


# ***PART:1***

# **PySpark Setup**

In [ ]:
# Install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download Spark 3.4.1 with Hadoop 3.3 from a working Apache archive
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

# Extract Spark
!tar -xzf spark-3.4.1-bin-hadoop3.tgz

# Install findspark
!pip install -q findspark

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

# Initialize findspark
import findspark
findspark.init()


In [ ]:
!ls -l spark-3.4.1-bin-hadoop3.tgz


-rw-r--r-- 1 root root 388341449 Jun 19  2023 spark-3.4.1-bin-hadoop3.tgz


# **Step 1: Prepare Spark + Scala and Load Data and Convert Timestamps**

***1. Create SparkSession***

This line initializes a Spark session, which is required to work with DataFrames in Spark. The session is named "Amazon Reviews - Q2 2014 Filter".

getOrCreate() checks if there is already a Spark session running. If not, it creates a new one.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Amazon Reviews - Q2 2014 Filter") \
    .getOrCreate()

In [ ]:
# # ✅ Enable Arrow for .toPandas() to work
# spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [ ]:
# # Disable Arrow (workaround for Colab compatibility issue)
# spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")

***2. Loading the Data***

This reads the CSV file located at the specified path into a DataFrame (df).

header=False: This means the CSV file does not have column headers in the first row.

inferSchema=True: This automatically tries to infer the data types (e.g., integer, string, etc.) for each column in the dataset.

In [ ]:
df = spark.read.csv("/content/drive/My Drive/MMU/Assignments/HPC/extracted_data/ratings_Electronics (1).csv",
                    header=False,
                    inferSchema=True)

***3. Assigning Column Names***

The CSV file you loaded doesn't have column names (since you set header=False). So, you're manually specifying the column names in a list: ["userId", "productId", "rating", "timestamp"].

The toDF(*columns) method is used to rename the columns of the DataFrame to the ones you just specified.

In [ ]:
# Add column names
columns = ["userId", "productId", "rating", "timestamp"]
df = df.toDF(*columns)

df.show(5)

+--------------+----------+------+----------+
|        userId| productId|rating| timestamp|
+--------------+----------+------+----------+
| AKM1MP6P0OYPR|0132793040|   5.0|1365811200|
|A2CX7LUOHB2NDG|0321732944|   5.0|1341100800|
|A2NWSAGRHCP8N5|0439886341|   1.0|1367193600|
|A2WNBOD3WNDNKT|0439886341|   3.0|1374451200|
|A1GI0U4ZRJA8WN|0439886341|   1.0|1334707200|
+--------------+----------+------+----------+
only showing top 5 rows



***4. Converting Unix Timestamps to Dates***

from_unixtime(col("timestamp")): This function converts the Unix timestamp (timestamp column) into a human-readable date format.

.withColumn("date", ...): Adds a new column named "date" to the DataFrame, which contains the human-readable dates corresponding to the Unix timestamps.

In [ ]:
from pyspark.sql.functions import from_unixtime, col

df = df.withColumn("date", from_unixtime(col("timestamp")))

***5. Displaying Timestamp and Date***

In [ ]:
df.select("timestamp", "date").show(5)

+----------+-------------------+
| timestamp|               date|
+----------+-------------------+
|1365811200|2013-04-13 00:00:00|
|1341100800|2012-07-01 00:00:00|
|1367193600|2013-04-29 00:00:00|
|1374451200|2013-07-22 00:00:00|
|1334707200|2012-04-18 00:00:00|
+----------+-------------------+
only showing top 5 rows



# ***Step 2: Preprocess Timestamps and Filter for Q2 2014***

***Code to Filter Data for Q2 2014***

Convert the Unix timestamps to human-readable dates (which you already did).

Filter the data to only include reviews from Q2 2014 (i.e., from April 1, 2014, to June 30, 2014).

In [ ]:
# Filter data for Q2 2014 (April 1, 2014 to June 30, 2014)
dfQ2 = df.withColumn("date", from_unixtime(col("timestamp"), "yyyy-MM-dd")) \
         .filter(col("date").between("2014-04-01", "2014-06-30"))

# Show the first 5 rows after filtering
dfQ2.show(5)

+--------------+----------+------+----------+----------+
|        userId| productId|rating| timestamp|      date|
+--------------+----------+------+----------+----------+
|A1QGNMC6O1VW39|0511189877|   5.0|1397433600|2014-04-14|
|A3J3BRHTDRFJ2G|0511189877|   2.0|1397433600|2014-04-14|
| AZYNQZ94U6VDB|0511189877|   5.0|1401321600|2014-05-29|
| AYTBGUX49LF3W|0528881469|   4.0|1398470400|2014-04-26|
|A3ILG56NE5QU37|0594033896|   5.0|1399420800|2014-05-07|
+--------------+----------+------+----------+----------+
only showing top 5 rows



# ***Step 3: Categorizing Products Based on Ratings.***

***Code to Categorize Products***

Group the data by productId.

Check if a product has any rating of ≥ 3.

Create two groups:

Group 1: Products with at least one rating ≥ 3.

Group 2: Products with all ratings < 3.

Explanation:
groupBy("productId")

This groups the data by productId to ensure you categorize products individually.

max(when(col("rating") >= 3, 1).otherwise(0))

This checks if any rating for a given product is ≥ 3.

when(col("rating") >= 3, 1) means if a rating is ≥ 3, we assign a 1, otherwise, we assign a 0.

max(...): Takes the maximum value for each product — if there’s any 1 (i.e., a rating ≥ 3), the result will be 1, indicating the product belongs to Group 1. Otherwise, it will be 0, meaning the product is in Group 2.

withColumn("group", when(col("has_high_rating") == 1, "Group1").otherwise("Group2"))

Based on the value of has_high_rating, it assigns the product to either Group 1 or Group 2.

productGroups.show(5)

This displays the first 5 rows of the categorized products so you can verify the results.

In [ ]:
from pyspark.sql.functions import max, when

# Group by productId and check if any rating >= 3
productGroups = dfQ2.groupBy("productId") \
    .agg(
        max(when(col("rating") >= 3, 1).otherwise(0)).alias("has_high_rating")
    ) \
    .withColumn("group", when(col("has_high_rating") == 1, "Group1").otherwise("Group2"))

# Show the first 5 rows of categorized products
productGroups.show(5)

+----------+---------------+------+
| productId|has_high_rating| group|
+----------+---------------+------+
|7793224531|              1|Group1|
|9966694242|              1|Group1|
|9967222247|              1|Group1|
|B000000O3J|              1|Group1|
|B00000J0IV|              1|Group1|
+----------+---------------+------+
only showing top 5 rows



# ***Step 4: Calculating the Number of Reviews per Product.***

***Code to Calculate Review Counts***

Group the data by productId and calculate how many reviews each product has received.

Join the review count with the product categorization (Group 1 and Group 2).

In [ ]:
 # Count the number of reviews per product
reviewCounts = dfQ2.groupBy("productId") \
    .agg(count("*").alias("review_count"))

# Join the review counts with the categorized products
finalData = productGroups.join(reviewCounts, "productId")

# Show the first 5 rows of the final data
finalData.show(5)

+----------+---------------+------+------------+
| productId|has_high_rating| group|review_count|
+----------+---------------+------+------------+
|7793224531|              1|Group1|           1|
|9966694242|              1|Group1|           2|
|9967222247|              1|Group1|           1|
|B000000O3J|              1|Group1|           1|
|B00000J0IV|              1|Group1|           1|
+----------+---------------+------+------------+
only showing top 5 rows



In [ ]:
# Check for products with only 1 review (edge case)
single_review_products = dfQ2.groupBy("productId").agg(count("*").alias("count")).filter(col("count") == 1)
print(f"Products with only 1 review: {single_review_products.count()}")

Products with only 1 review: 66200


In [ ]:
low_review_distribution = dfQ2.groupBy("productId").agg(count("*").alias("count")) \
                             .filter(col("count") <= 5) \
                             .groupBy("count").agg(count("*").alias("product_count")) \
                             .orderBy("count") \
                             .withColumnRenamed("count", "review_count")

low_review_distribution.show()

+------------+-------------+
|review_count|product_count|
+------------+-------------+
|           1|        66200|
|           2|        18740|
|           3|         9061|
|           4|         5464|
|           5|         3537|
+------------+-------------+



***Step for: Mann-Whitney U test***

In [ ]:
from scipy.stats import mannwhitneyu

# Perform Mann-Whitney U test
stat, p_value = mannwhitneyu(group1_reviews, group2_reviews)
print(f"Mann-Whitney U test p-value: {p_value:.4f}")

if p_value < 0.05:
    print("Statistically significant difference between groups.")
else:
    print("No statistically significant difference between groups.")

Mann-Whitney U test p-value: 0.0000
Statistically significant difference between groups.


***Step for: Graph***

In [ ]:
import numpy as np

group1_log = np.log1p(group1_reviews)  # log(1 + x) to handle zeros
group2_log = np.log1p(group2_reviews)

In [ ]:
print(f"Group 1 median: {np.median(group1_reviews)}, Group 2 median: {np.median(group2_reviews)}")

Group 1 median: 2.0, Group 2 median: 1.0


In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x=['Group 1']*len(group1_reviews) + ['Group 2']*len(group2_reviews),
            y=list(group1_reviews) + list(group2_reviews),
            palette=['skyblue', 'salmon'])
sns.swarmplot(x=['Group 1']*len(group1_reviews) + ['Group 2']*len(group2_reviews),
              y=list(group1_reviews) + list(group2_reviews),
              color='black', alpha=0.3, size=3)  # Optional: show all points
plt.yscale('log')
plt.ylabel('Review Count (log scale)')
plt.title('Review Count Distribution: Group 1 vs. Group 2')
plt.show()

<ipython-input-126-41ae5b6fab9f>:2: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x=['Group 1']*len(group1_reviews) + ['Group 2']*len(group2_reviews),


In [ ]:
plt.figure(figsize=(10, 6))
sns.ecdfplot(data=group1_reviews, label='Group 1', color='skyblue')
sns.ecdfplot(data=group2_reviews, label='Group 2', color='salmon')
plt.xlabel('Review Count')
plt.ylabel('Proportion of Users')
plt.title('Cumulative Distribution of Reviews')
plt.legend()
plt.show()

# ***Step 5: Perform Statistical Test (t-test)***

***Code to Perform the T-Test (Using Pandas and SciPy)***

You’ll extract the review_count values for both Group 1 and Group 2.

Perform a t-test to check if the difference in the number of reviews per product between the two groups is statistically significant.

The t-statistic (21.62) is very high, indicating that this difference is really large.

The p-value is extremely small (0), which means the difference in reviews between the two groups is statistically significant — meaning we can be almost certain that this difference is not due to random chance.

Below test shows that products with higher ratings (≥ 3) tend to have significantly more reviews than those with lower ratings (< 3). This difference is real and not just a random occurrence.

In [ ]:
from pyspark.sql.functions import avg, stddev, count

# Step 1: Compute group stats in Spark
group_stats = finalData.groupBy("group").agg(
    count("review_count").alias("n"),
    avg("review_count").alias("mean"),
    stddev("review_count").alias("stddev")
)

group_stats.show()

+------+------+-----------------+------------------+
| group|     n|             mean|            stddev|
+------+------+-----------------+------------------+
|Group2| 14735|1.141364099083814|0.5388742029934758|
|Group1|107281|6.032717815829457|27.467639281255813|
+------+------+-----------------+------------------+



In [ ]:
# Collect data from Spark to Python
data = finalData.select("group", "review_count").collect()

# Convert to a Pandas DataFrame manually
import pandas as pd
finalData_pd = pd.DataFrame(data, columns=["group", "review_count"])

In [ ]:
# Separate the data into two groups
group1_reviews = finalData_pd[finalData_pd['group'] == 'Group1']['review_count']
group2_reviews = finalData_pd[finalData_pd['group'] == 'Group2']['review_count']

# Perform t-test

t_stat, p_value = stats.ttest_ind(group1_reviews, group2_reviews)

# Display the t-statistic and p-value
print(f"T-statistic: {t_stat}, P-value: {p_value}")

T-statistic: 21.615724227746018, P-value: 2.0001809061110027e-103


# ***Test I Used***

---

### 🔵 My Original Hypothesis:
> "In the second quarter of 2014, products given a review rating of 3 or more are significantly different compared to other products."

### 🟢 Problem with the original hypothesis:
- It’s **vague**:  
  - "Significantly different" — **different in what way?** (Sales? Ratings? Number of reviews? Nothing specific.)
- It’s **unclear** what the measurable metric is.
- "Products given a rating of 3 or more" — **individual reviews** are considered, but I need to decide at the **product level**.

---

### 🟢 Refined Hypothesis I am testing:
### 🔵 Clarification on Product-Level Aggregation:
- **Group 1**: Products with **at least one rating ≥3**.  
- **Group 2**: Products with **all ratings <3**.  
- This ensures the hypothesis is tested at the **product level**, not individual reviews.  

✅ This refined hypothesis is:
- **Clear**: I am comparing the **number of reviews** per product.
- **Measurable**: I have the `review_count` metric.
- **Actionable**: It divides products into two groups and compares them.

---

### ⚡ Which statistical test should I use?

**I am using an Independent Two-Sample t-test** (also called an **unpaired t-test**) because:
- I have **two independent groups** (Group 1 and Group 2).
- I am comparing their **means** (average number of reviews).
- I want to know if the difference between means is statistically significant.

---

### 🧠 Summary Table:

| Aspect                     | Original Hypothesis                                          | Refined Hypothesis                                              |
|-----------------------------|--------------------------------------------------------------|------------------------------------------------------------------|
| Clarity                     | Vague ("significantly different" is unclear)                | Clear ("review count difference")                               |
| Measurement                 | Not specified                                               | Number of reviews per product                                   |
| Testing Approach            | Not clear                                                   | Two-Sample Independent **t-test**                               |
| Groups                      | Products with reviews ≥3 vs others (but not defined properly) | Products with at least one rating ≥3 vs. all ratings <3          |

---

### 🔥 So Final Answer:
- **Statistical Test Used:** **Two-Sample Independent t-test** (comparing means of two independent groups)
- **Why:** To check if the **mean number of reviews per product** is significantly different between the two groups.

---

# ***Limitations***

### 🔴 Limitations and Future Work:
1. **Causation vs. Correlation**:  
   - Review count ≠ product popularity (no sales data).  
2. **Metadata Gaps**:  
   - Missing product categories/pricing to control for confounding variables.  
3. **Temporal Bias**:  
   - Only Q2 2014 data analyzed; trends may vary annually.  
4. **Non-Normal Data**:  
   - Review counts are right-skewed; Mann-Whitney U test may be more robust.  

**Future Work**:  
- Include sentiment analysis of reviews.  
- Compare with sales/price data if available.  